* basic roberta ft: 0.6589791487657798 (thr 0.3)
* basic roberta ft (head first): 0.6768011808573329 (thr 0.42)
* fine tune roberta on weird clf, then only head on spans, then whole: 0.6853127403287083 (thr 0.32)
* 

In [1]:
from transformers import RobertaTokenizer, RobertaForTokenClassification
from transformers import BertTokenizer, BertForTokenClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

In [2]:
import numpy as np
import pandas as pd

In [3]:
model_name = 'roberta-base'  #roberta-base
saved_name = 'models/roberta_single'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [32]:
model = AutoModelForTokenClassification.from_pretrained(model_name)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able

In [5]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

In [6]:
labels = torch.tensor([1] * inputs["input_ids"].size(1)).unsqueeze(0)  # Batch size 1

In [7]:
outputs = model(**inputs, labels=labels)

In [8]:
outputs

TokenClassifierOutput(loss=tensor(3.5819, grad_fn=<NllLossBackward>), logits=tensor([[[ 4.4480, -3.6460],
         [ 3.0508, -2.3449],
         [ 3.4750, -2.5897],
         [ 1.7331, -0.5390],
         [-0.1971,  1.1977],
         [-0.9398,  1.8006],
         [-2.1010,  2.9615],
         [ 3.6444, -2.7873]]], grad_fn=<AddBackward0>), hidden_states=None, attentions=None)

# Create labels for tagging

In [5]:
import os
import numpy as np
import pandas as pd
from ast import literal_eval
import re
import nltk
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize

path = 'data/'

In [6]:
trial = pd.read_csv(path + 'tsd_trial.csv')
train = pd.read_csv(path + 'tsd_train.csv')
final_test = pd.read_csv(path + 'tsd_test.csv')

train['spans'] = train.spans.apply(literal_eval)
trial['spans'] = trial.spans.apply(literal_eval)
trial.shape, train.shape, final_test.shape

((690, 2), (7939, 2), (2000, 1))

In [7]:
print(len(set(trial.text).intersection(set(train.text))))
print(len(set(final_test.text).intersection(set(train.text))))

8
0


In [8]:
print((train.spans.apply(len) == 0).mean())
print((trial.spans.apply(len) == 0).mean())

0.06109081748331024
0.06231884057971015


In [9]:
import spans_utils
from importlib import reload
reload(spans_utils)
from spans_utils import display_spans, spans2labels, labels2spans

display_spans(trial.spans[0], trial.text[0])
display_spans(trial.spans[0], trial.text[0])

In [10]:
from tqdm.auto import tqdm, trange

In [11]:
n = 0
for row in tqdm([row for i, row in trial.iterrows()]):
    labels = spans2labels(row.text, row.spans, tokenizer)
    spans2 = labels2spans(row.text, labels, tokenizer)
    if row.spans != spans2:
        t = row.text.replace(' ', '+')
        display_spans(row.spans, t)
        display_spans(spans2, t)
        n += 1
print(n)

http://www.politicususa.com/2016/09/02/trump-craven-maniacal-david-bossie-clinton-character-assassin.html

New+Trump+Hire+Proves+Hillary+Clinton’s+Vast+Right+Wing+Conspiracy+is+Real
Trump's+new+Deputy+Campaign+Manager,+Citizens+United+President+David+Bossie,+"has+devoted+his+career+ever+since+to+trying+to+tear+down+Hillary+Clinton"

There+is+no+disguising+the+truth:+Bossie+will+be+assisting+Kellyanne+Conway+as+an+anti-Clinton+character+assassin,+and+Hillary+for+America+Chair+John+Podesta+released+the+following+statement:

“David+Bossie+is+so+craven+and+maniacal+that+in+the+heyday+of+the+overreaching,+Gingrich-era+Congress,+the+top+Whitewater+conspiracy+theorist+in+the+House+had+to+fire+him+for+doctoring+evidence.+He+has+devoted+his+career+ever+since+to+trying+to+tear+down+Hillary+Clinton

Another+liar+just+like+Trump.+I+think+only+<b>idiots</b>..............
http://www.politicususa.com/2016/09/02/trump-craven-maniacal-david-bossie-clinton-character-assassin.html

New+Trump+Hire+Proves+


39


In [12]:
train_labels = [spans2labels(row.text, row.spans, tokenizer) for i, row in tqdm(train.iterrows())]

In [13]:
trial_labels = [spans2labels(row.text, row.spans, tokenizer) for i, row in tqdm(trial.iterrows())]

In [14]:
train['labels'] = train_labels
trial['labels'] = trial_labels

In [15]:
class SpansDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [16]:
train_dataset = SpansDataset(tokenizer(train.text.tolist()), train_labels)
eval_dataset = SpansDataset(tokenizer(trial.text.tolist()), trial_labels)

In [17]:
final_test_dataset = SpansDataset(tokenizer(final_test.text.tolist()))

In [18]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer, padding=True)

In [19]:
import numpy as np
from semeval2021 import f1

# Train a single-task model

https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb
https://huggingface.co/transformers/custom_datasets.html

In [21]:
from transformers import Trainer, TrainingArguments
from transformers.file_utils import cached_property
from typing import Tuple

device = torch.device('cuda:3')

class TrAr(TrainingArguments):
    @cached_property
    def _setup_devices(self) -> Tuple["torch.device", int]:
        return device

In [22]:
torch.cuda.set_device(device)

In [21]:
model.to(device);

NameError: name 'model' is not defined

In [41]:
for param in model.roberta.parameters():
    param.requires_grad = True

In [64]:
training_args = TrAr(
    output_dir='./models/roberta_single_v2',   # output directory
    overwrite_output_dir=True,
    num_train_epochs=10,            # total # of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=3000,               # number of warmup steps for learning rate scheduler
    weight_decay=1e-8,              # strength of weight decay
    learning_rate=1e-5,
    logging_dir='./logs',           # directory for storing logs
    logging_steps=500,
    eval_steps=500,
    evaluation_strategy='steps',
)

In [65]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset,           # evaluation dataset
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [57]:
training_args.device

device(type='cuda', index=3)

* The minimal loss of a single-task model (full) was about 28% on validation with 0.04 on train. 
* If we first train only head (batch 8, lr 1e-3 with 3K warmup and 1e-8 decline), we get minimal loss of 0.185 on validation with 0.23 on train
* Training then the whole model (batch 8, lr 1e-5 with 3K warmup and 1e-8 decline) we get minimal loss of 0.175 on validation with 0.21 on train

In [58]:
trainer.train()

Step,Training Loss,Validation Loss
500,0.200961,0.177887
1000,0.220103,0.177245
1500,0.215706,0.177792
2000,0.210133,0.175849
2500,0.208366,0.187829
3000,0.207653,0.193692
3500,0.194796,0.189296
4000,0.184825,0.206708
4500,0.157858,0.200036
4673,0.157858,0.216965


KeyboardInterrupt: 

In [62]:
model = model.from_pretrained('models/roberta_single_v2/checkpoint-2000')

In [68]:
model = model.save_pretrained('models/roberta_single_v2')

In [67]:
trainer.evaluate()

{'eval_loss': 0.17584852874279022}

### evaluate

In [70]:
pred = trainer.predict(eval_dataset)

In [71]:
for threshold in [0, 0.01, 0.03, 0.1, 0.3, 0.4, 0.5, 0.6, 0.7, 1]:
    preds = []
    for text, pr in zip(trial.text, pred.predictions):
        proba = np.exp(pr[pr[:, 0]!=-100])
        proba /= proba.sum(axis=1, keepdims=True)
        labels = (proba[:, 1] >= threshold).astype(int).tolist()
        preds.append(labels2spans(text, labels, tokenizer))
    print(threshold, np.mean([f1(p, y) for p, y in zip(preds, trial.spans)]))

0 0.21345926240649193
0.01 0.25205437406740794
0.03 0.37667972240631
0.1 0.5772878732277155
0.3 0.66359735406299
0.4 0.6731999549591532
0.5 0.6702516698368454
0.6 0.6618129018990354
0.7 0.6474321946754974
1 0.06231884057971015


In [73]:
for threshold in [0.3, 0.32, 0.35, 0.38, 0.4, 0.42, 0.45, 0.5, 0.55, 0.6]:
    preds = []
    for text, pr in zip(trial.text, pred.predictions):
        proba = np.exp(pr[pr[:, 0]!=-100])
        proba /= proba.sum(axis=1, keepdims=True)
        labels = (proba[:, 1] >= threshold).astype(int).tolist()
        preds.append(labels2spans(text, labels, tokenizer))
    print(threshold, np.mean([f1(p, y) for p, y in zip(preds, trial.spans)]))

0.3 0.66359735406299
0.32 0.6680702150590394
0.35 0.6700635517847642
0.38 0.6735559287333039
0.4 0.6731999549591532
0.42 0.6768011808573329
0.45 0.6729071697382535
0.5 0.6702516698368454
0.55 0.6634497922801099
0.6 0.6618129018990354


## Prepare a submission

In [76]:
pred = trainer.predict(final_test_dataset)

In [84]:
threshold = 0.35
preds = []
for text, pr in zip(final_test.text, pred.predictions):
    proba = np.exp(pr[pr[:, 0]!=-100])
    proba /= proba.sum(axis=1, keepdims=True)
    labels = (proba[:, 1] >= threshold).astype(int).tolist()
    preds.append(labels2spans(text, labels, tokenizer))

In [93]:
row = final_test.sample(1).iloc[0]
display_spans(preds[row.name], row.text)

In [95]:
with open("results/roberta_th03.txt", "w") as out:
    for i in range(len(preds)):
        out.write(f"{str(i)}\t{str(preds[i])}\n")


# M
# Mult
# Multitasking

In [23]:
from transformers import RobertaTokenizer, RobertaForTokenClassification, RobertaForSequenceClassification
from transformers import BertTokenizer, BertForTokenClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

In [24]:
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.modeling_outputs import SequenceClassifierOutput
import torch.nn as nn
from torch.nn import CrossEntropyLoss, MSELoss

class WMean(nn.Module):
    def __init__(self, dim=-2):
        super(WMean, self).__init__()
        self.pow = torch.nn.Parameter(data=torch.Tensor([1.0]), requires_grad=True)
        self.coef = torch.nn.Parameter(data=torch.Tensor([0.0, 1.0]), requires_grad=True)
        self.dim = dim
    
    def forward(self, x, mask=None):
        result = x ** self.pow[0]
        if mask is None:
            mp = result.mean(dim=-1)
        else:
            mp = (result * mask).sum(dim=self.dim) / mask.sum(dim=self.dim)
        return torch.log(mp) * self.coef[1] + self.coef[0]


class RobertaTaggerClassifier(RobertaForTokenClassification):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.roberta = RobertaModel(config, add_pooling_layer=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        self.wmean = WMean()

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]

        sequence_output = self.dropout(sequence_output)
        token_logits = self.classifier(sequence_output)

        if attention_mask is not None:
            masks = attention_mask.unsqueeze(-1).repeat(1, 1, 2)
        else:
            masks = None

        logits = self.wmean(torch.softmax(token_logits, dim=-1), mask=masks)

        loss = None
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [6]:
rtt_model = RobertaTaggerClassifier.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaTaggerClassifier: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaTaggerClassifier from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaTaggerClassifier from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaTaggerClassifier were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias', 'wmean.pow', 'wmean.coef']
You should probably TRAIN this model on a down-stream task to be a

In [7]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [8]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

In [9]:
rtt_model(**inputs)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.7042, -0.6822]], grad_fn=<AddBackward0>), hidden_states=None, attentions=None)

In [26]:
import pandas as pd

In [27]:
df1 = pd.read_csv('../data/train/train.1.tsv', sep='\t')
df0 = pd.read_csv('../data/train/train_small.0.tsv', sep='\t')
df01 = pd.concat([df1, df0], ignore_index=True)
df01.label = df01.label.astype(int)
print(df01.shape)
df01.sample(3)

(358984, 2)


,comment_text,label
272706,Those aren't anti fascists those are national ...,0
108545,Sherman is a traitor. A running dog lackey to ...,1
343175,"""\n\nGuy Chapman Slips up\n\n""""Actually Wessel...",0


In [28]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df01, test_size=0.1, random_state=1)

In [29]:
df_train.head(10)

,comment_text,label
214726,"My Dad had a saying, if you can't play, play l...",0
275434,Awesome! I realize players cannot / will not s...,0
315281,"If it were a school, I'd fire the principal fo...",0
225573,Do you believe POTUS should stop having he and...,0
214259,The Liberals better wake up. \n\nRemember wha...,0
274177,This legislature is violating the Alaska State...,0
20450,if a trip wire or sensors could be hooked up t...,1
182019,See what Dr. Fred Baughman had to say about dr...,0
317139,Impatient millinial. Is that something to adve...,0
23614,"So ridiculous people still call this ""Our oil""...",1


In [14]:
class SpansDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [30]:
clf_train_dataset = SpansDataset(
    tokenizer(df_train.comment_text.tolist(), truncation=True), 
    df_train.label.tolist()
)

In [31]:
clf_test_dataset = SpansDataset(
    tokenizer(df_test.comment_text.tolist(), truncation=True), 
    df_test.label.tolist()
)

In [32]:
clf_test_small_dataset = SpansDataset(
    tokenizer(df_test.comment_text.iloc[:3000].tolist(), truncation=True), 
    df_test.label[:3000].tolist()
)

In [39]:
device = torch.device('cuda:3')

from transformers import Trainer, TrainingArguments
from transformers.file_utils import cached_property
from typing import Tuple

class TrAr(TrainingArguments):
    @cached_property
    def _setup_devices(self) -> Tuple["torch.device", int]:
        return device, 1

The strategy: first tune the head only with large batches and LR, then tune the whole model. 

In [40]:
for param in rtt_model.roberta.parameters():
    param.requires_grad = True

In [54]:
training_args = TrAr(
    output_dir='./models/roberta_clf',   # output directory
    overwrite_output_dir=True,
    num_train_epochs=3,             # total # of training epochs
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=64,  # batch size for evaluation
    warmup_steps=3000,              # number of warmup steps for learning rate scheduler
    weight_decay=1e-7,              # strength of weight decay
    learning_rate=1e-6,
    logging_dir='./logs',           # directory for storing logs
    logging_steps=100,
    eval_steps=500,
    evaluation_strategy='steps',
)

In [55]:
trainer = Trainer(
    model=rtt_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                      # training arguments, defined above
    train_dataset=clf_train_dataset,         # training dataset
    eval_dataset=clf_test_small_dataset,           # evaluation dataset
    tokenizer=tokenizer,
)

In [67]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [64]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [59]:
print(rtt_model.wmean.pow)
print(rtt_model.wmean.coef)

Parameter containing:
tensor([1.6784], device='cuda:3', requires_grad=True)
Parameter containing:
tensor([0.4055, 1.6780], device='cuda:3', requires_grad=True)


In [61]:
rtt_model.save_pretrained('models/roberta_clf_proba')

In [62]:
rtt_model = None

# Fine tune the averager classifier

In [23]:
model = AutoModelForTokenClassification.from_pretrained('models/roberta_clf_proba')

Some weights of the model checkpoint at models/roberta_clf_proba were not used when initializing RobertaForTokenClassification: ['wmean.pow', 'wmean.coef']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [107]:
training_args = TrAr(
    output_dir='./models/roberta_clf_ft',   # output directory
    overwrite_output_dir=True,
    num_train_epochs=30,            # total # of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=3000,               # number of warmup steps for learning rate scheduler
    weight_decay=1e-8,              # strength of weight decay
    learning_rate=1e-6,
    logging_dir='./logs',           # directory for storing logs
    logging_steps=500,
    eval_steps=500,
    evaluation_strategy='steps',
)

In [108]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset,           # evaluation dataset
    data_collator=data_collator,
    tokenizer=tokenizer,
)

* the raw quasi-classifier: no use in the model at all
* fine tuned head: still no use, the best score is 0.2138
* fine tune whole model: 0.3 0.6849391042415774

In [116]:
pred = trainer.predict(eval_dataset)
for threshold in [0, 0.01, 0.03, 0.1, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7, 1]:
    preds = []
    for text, pr in zip(trial.text, pred.predictions):
        proba = np.exp(pr[pr[:, 0]!=-100])
        proba /= proba.sum(axis=1, keepdims=True)
        labels = (proba[:, 1] >= threshold).astype(int).tolist()
        preds.append(labels2spans(text, labels, tokenizer))
    print(threshold, np.mean([f1(p, y) for p, y in zip(preds, trial.spans)]))

0 0.21345926240649193
0.01 0.33754217278281196
0.03 0.5097796783203596
0.1 0.6443839746429902
0.25 0.6813249614759649
0.3 0.6849391042415774
0.35 0.684180917165571
0.4 0.6787335180780203
0.5 0.6640287939316257
0.6 0.6514593408593448
0.7 0.6172986482655661
1 0.06231884057971015


In [117]:
for threshold in [ 0.25, 0.28, 0.3, 0.32, 0.35]:
    preds = []
    for text, pr in zip(trial.text, pred.predictions):
        proba = np.exp(pr[pr[:, 0]!=-100])
        proba /= proba.sum(axis=1, keepdims=True)
        labels = (proba[:, 1] >= threshold).astype(int).tolist()
        preds.append(labels2spans(text, labels, tokenizer))
    print(threshold, np.mean([f1(p, y) for p, y in zip(preds, trial.spans)]))

0.25 0.6813249614759649
0.28 0.6825281648777168
0.3 0.6849391042415774
0.32 0.6853127403287083
0.35 0.684180917165571


In [104]:
for param in model.roberta.parameters():
    param.requires_grad = True

If we fine tune only the classification head, we achieve the same validation loss (.29) as when fine tuning the whole model, without overfitting at all! However, at this value the model just stops learning. 

If we start tuning the whole model, validation loss goes to 0.2 quickly, and the learning stops with validation loss around 0.17 and train loss around 0.03

In [110]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [111]:
trainer.train()

Step,Training Loss,Validation Loss
500,0.304273,0.249575
1000,0.296361,0.221189
1500,0.270973,0.194118
2000,0.237798,0.182866
2500,0.231155,0.177148
3000,0.246954,0.175022
3500,0.225380,0.173276
4000,0.233015,0.175608
4500,0.229856,0.174285
5000,0.232127,0.166275


KeyboardInterrupt: 

In [114]:
model.save_pretrained('models/roberta_clf_ft')

In [292]:
pred = trainer.predict(final_test_dataset)

In [293]:
threshold = 0.6
preds = []
for text, pr in zip(final_test.text, pred.predictions):
    proba = np.exp(pr[pr[:, 0]!=-100])
    proba /= proba.sum(axis=1, keepdims=True)
    labels = (proba[:, 1] >= threshold).astype(int).tolist()
    preds.append(labels2spans(text, labels, tokenizer))
print(len(preds))

2000


In [171]:
with open("results/roberta_ft_th06.txt", "w") as out:
    for i in range(len(preds)):
        out.write(f"{str(i)}\t{str(preds[i])}\n")

In [294]:
row = final_test.sample(1).iloc[0]
display_spans(preds[row.name], row.text)

#### try to improve the decoding

In [310]:
logits = pred.predictions[row.name]
proba = np.exp(logits[logits[:, 0]!=-100])
proba /= proba.sum(axis=1, keepdims=True)

In [290]:
len(tokenizer.truncate_sequences(list(range(1000)), truncation_strategy='longest_first')[0])

1000

In [298]:
def decode_spans(text, target_proba, threshold, tokenizer, agg=max, space='Ġ', punct='.,?!()', return_spans=True, truncate=True):
    # try to label a whole multitoken word consistently
    punct = set(punct)
    result = []
    token_labels = [0]
    token_ids = tokenizer.encode(text)
    if truncate:
        token_ids = token_ids[:tokenizer.model_max_length]
    tokens = tokenizer.convert_ids_to_tokens(token_ids)
    assert len(tokens) <= len(target_proba), '{} > {}'.format(len(tokens), len(target_proba))
    left = 0
    prev_label = 0
    word_start = 0
    word_scores = []
    for i, (tok, tp) in enumerate(zip(tokens[1:], target_proba[1:])):
        right = left + len(tok)
        # start of word (except the first one)
        if tok[0] == space or tok == '</s>' or tok in punct:
            word_score = agg(word_scores)
            if word_score > threshold:
                if not prev_label:
                    word_start += 1
                prev_label = 1
                result.extend(range(word_start, left))
                token_labels.extend([1] * len(word_scores))
            else:
                prev_label = 0
                token_labels.extend([0] * len(word_scores))
            word_scores = []
            word_start = left
        word_scores.append(tp)
        left = right
    token_labels.extend([0] * len(word_scores))
    if return_spans:
        return result
    else:
        return token_labels


In [311]:
result = decode_spans(row.text, proba[:, 1], 0.4, tokenizer)
print(result)
display_spans(result, row.text)

[0.7628563] Ġthing Ġdamn 0.7628563
[330, 331, 332, 333]


```
agg     max score
no agg: 0.6849391042415774
max:    0.6733625076759546
min:    0.640940791436199 
mean:   0.6702291605163007
pindep: 0.6529080645860461
```

In [338]:
def pindep(arr):
    pos = np.prod(arr)
    neg = np.prod([1 - x for x in arr])
    return pos / (pos + neg)

In [339]:
pred = trainer.predict(eval_dataset)

for threshold in [0, 0.01, 0.03, 0.1, 0.25, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7, 1]:
    preds = []
    for text, pr in zip(trial.text, pred.predictions):
        proba = np.exp(pr[pr[:, 0]!=-100])
        proba /= proba.sum(axis=1, keepdims=True)
        labels = (proba[:, 1] >= threshold).astype(int).tolist()
        preds.append(labels2spans(text, labels, tokenizer))
    preds2 = []
    for text, pr in zip(trial.text, pred.predictions):
        proba = np.exp(pr[pr[:, 0]!=-100])
        proba /= proba.sum(axis=1, keepdims=True)
        spans = decode_spans(text, proba[:, 1], threshold, tokenizer, agg=pindep)
        preds2.append(spans)
    print(
        threshold, 
        np.mean([f1(p, y) for p, y in zip(preds, trial.spans)]),
        np.mean([f1(p, y) for p, y in zip(preds2, trial.spans)]),
    )

0 0.21345926240649193 0.2129121719490078
0.01 0.33754217278281196 0.3577009491893737
0.03 0.5097796783203596 0.5219355909995085
0.1 0.6443839746429902 0.6220979241154265
0.25 0.6813249614759649 0.6506613818412771
0.3 0.6849391042415774 0.6529080645860461
0.35 0.684180917165571 0.6496544187202091
0.4 0.6787335180780203 0.6423737170064413
0.5 0.6640287939316257 0.6289225858545294
0.6 0.6514593408593448 0.6138640904113093
0.7 0.6172986482655661 0.5890367232017543
1 0.06231884057971015 0.06231884057971015


# Auto-label the toxic dataset

In [34]:
kaggle_toxic = pd.read_csv('../data/train/train.1.tsv', sep='\t')

In [35]:
kaggle_dataset = SpansDataset(
    tokenizer(kaggle_toxic.comment_text.tolist(), truncation=True), 
)

In [36]:
print(len(kaggle_dataset))

158984


In [25]:
model = AutoModelForTokenClassification.from_pretrained('models/roberta_clf_ft')
model.to(device);

In [30]:
training_args = TrAr(
    output_dir='./models/roberta_clf_ft',   # output directory
    overwrite_output_dir=True,
    num_train_epochs=30,            # total # of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=4,   # smaller batch sizes are actually better, because seq len per batch gets shorter
    warmup_steps=3000,               # number of warmup steps for learning rate scheduler
    weight_decay=1e-8,              # strength of weight decay
    learning_rate=1e-6,
    logging_dir='./logs',           # directory for storing logs
    logging_steps=500,
    eval_steps=500,
    evaluation_strategy='steps'
)

In [31]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset,           # evaluation dataset
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [37]:
kaggle_preds = trainer.predict(kaggle_dataset)

In [38]:
kaggle_preds.predictions.shape

(158984, 512, 2)

In [39]:
from tqdm.auto import tqdm, trange

In [40]:
import spans_utils
reload(spans_utils)
from spans_utils import decode_spans

In [41]:
kaggle_proba = []
kaggle_labels = []
for i in trange(len(kaggle_dataset)):
    n = len(kaggle_dataset[i]['input_ids'])
    logits = kaggle_preds.predictions[i][:n].T
    p = np.exp(logits)
    p /= p.sum(axis=0)
    kaggle_proba.append(p[1])
    text = kaggle_toxic.comment_text[i]
    token_labels = decode_spans(text, p[1], 0.33, tokenizer, return_spans=False, agg=max)
    kaggle_labels.append(token_labels)

Token indices sequence length is longer than the specified maximum sequence length for this model (737 > 512). Running this sequence through the model will result in indexing errors


In [42]:
maxes = pd.Series(max(p) for p in kaggle_proba)
maxes.describe()

count    158984.000000
mean          0.753992
std           0.190560
min           0.022448
25%           0.647929
50%           0.798888
75%           0.904961
max           0.998292
dtype: float64

In [43]:
print((maxes>0.33).mean())
print((maxes>0.50).mean())

0.9608451164897096
0.8875295627232929


In [44]:
kaggle_dataset.labels = kaggle_labels

In [52]:
row = kaggle_toxic.sample(1).iloc[0]
spans = labels2spans(text=row.comment_text, labels=kaggle_dataset[row.name]['labels'], tokenizer=tokenizer)
display_spans(spans, row.comment_text)

i = row.name
n = len(kaggle_dataset[i]['input_ids'])
logits = kaggle_preds.predictions[i][:n].T
p = np.exp(logits)
p /= p.sum(axis=0)

token_labels = decode_spans(row.comment_text, p[1], 0.33, tokenizer, return_spans=False, agg=max)
spans = labels2spans(labels=token_labels, text=row.comment_text, tokenizer=tokenizer)
display_spans(spans, row.comment_text)

In [56]:
training_args = TrAr(
    output_dir='./models/roberta_selflabel',   # output directory
    overwrite_output_dir=True,
    num_train_epochs=1,            # total # of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=4,   # smaller batch sizes are actually better, because seq len per batch gets shorter
    warmup_steps=3000,               # number of warmup steps for learning rate scheduler
    weight_decay=1e-8,              # strength of weight decay
    learning_rate=1e-7,
    logging_dir='./logs',           # directory for storing logs
    logging_steps=500,
    eval_steps=500,
    evaluation_strategy='steps',
    label_smoothing_factor=0.1,
)

In [57]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=kaggle_dataset,         # training dataset
    eval_dataset=eval_dataset,           # evaluation dataset
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [58]:
trainer.train()

Step,Training Loss,Validation Loss,Runtime,Samples Per Second
500,0.276700,0.321415,2.031800,339.598000
1000,0.270700,0.315323,2.017400,342.020000
1500,0.268700,0.312616,2.025300,340.692000
2000,0.264700,0.315503,2.010000,343.290000
2500,0.260000,0.319900,2.010300,343.239000
3000,0.257800,0.324110,2.010800,343.152000
3500,0.255300,0.326413,2.005600,344.041000
4000,0.254600,0.326755,2.010300,343.225000
4500,0.252100,0.327561,2.026000,340.568000
5000,0.250200,0.329431,2.007100,343.780000


KeyboardInterrupt: 

Best score is **0.6865**

In [60]:
pred = trainer.predict(eval_dataset)
for threshold in [0, 0.01, 0.03, 0.1, 0.25, 0.3, 0.35, 0.37, 0.4, 0.5, 0.6, 0.7, 1]:
    preds = []
    for text, pr in zip(trial.text, pred.predictions):
        proba = np.exp(pr[pr[:, 0]!=-100])
        proba /= proba.sum(axis=1, keepdims=True)
        labels = (proba[:, 1] >= threshold).astype(int).tolist()
        preds.append(labels2spans(text, labels, tokenizer))
    print(threshold, np.mean([f1(p, y) for p, y in zip(preds, trial.spans)]))

0 0.21345926240649193
0.01 0.21345926240649193
0.03 0.21447329483589261
0.1 0.6487386870668558
0.25 0.6793734526472192
0.3 0.6826507661661871
0.35 0.686369410349317
0.37 0.6865370507610563
0.4 0.6857396940615527
0.5 0.6865664238391493
0.6 0.6841674313771393
0.7 0.6857623157818409
1 0.06231884057971015


After tuning on the pseudo-labeled set, tune once more on the original set to calibrate the probabilities

In [74]:
device = torch.device('cuda:4')

class TrAr(TrainingArguments):
    @cached_property
    def _setup_devices(self) -> Tuple["torch.device", int]:
        return device

In [87]:
training_args = TrAr(
    output_dir='./models/roberta_selflabel_final',   # output directory
    overwrite_output_dir=True,
    num_train_epochs=2,            # total # of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=4,   # smaller batch sizes are actually better, because seq len per batch gets shorter
    warmup_steps=500,               # number of warmup steps for learning rate scheduler
    weight_decay=1e-8,              # strength of weight decay
    learning_rate=1e-6,
    logging_dir='./logs',           # directory for storing logs
    logging_steps=100,
    eval_steps=100,
    evaluation_strategy='steps',
    label_smoothing_factor=0,
)

In [88]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset,           # evaluation dataset
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [89]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [90]:
trainer.train()

Step,Training Loss,Validation Loss,Runtime,Samples Per Second
100,0.200000,0.165277,2.082800,331.284000
200,0.211100,0.165285,2.030000,339.908000
300,0.189600,0.169712,2.057100,335.431000
400,0.200900,0.170866,1.971600,349.972000
500,0.173400,0.165180,1.974800,349.402000
600,0.224800,0.163509,1.973600,349.617000
700,0.245400,0.160556,1.990600,346.629000
800,0.207200,0.164278,1.979400,348.588000
900,0.188100,0.170596,1.980000,348.491000
1000,0.210900,0.163308,1.974600,349.429000


TrainOutput(global_step=1986, training_loss=0.20756035917114996, metrics={'train_runtime': 235.7622, 'train_samples_per_second': 8.424, 'total_flos': 1529497023746184, 'epoch': 2.0})

max: **0.6887** (случайно потренировал на основной выборке с label smoothing 0.1)

In [93]:
pred = trainer.predict(eval_dataset)
for threshold in [0, 0.01, 0.03, 0.1, 0.25, 0.3, 0.35, 0.37, 0.4, 0.43, 0.5, 0.6, 0.7, 1]:
    preds = []
    for text, pr in zip(trial.text, pred.predictions):
        proba = np.exp(pr[pr[:, 0]!=-100])
        proba /= proba.sum(axis=1, keepdims=True)
        labels = (proba[:, 1] >= threshold).astype(int).tolist()
        preds.append(labels2spans(text, labels, tokenizer))
    print(threshold, np.mean([f1(p, y) for p, y in zip(preds, trial.spans)]))

0 0.21345926240649193
0.01 0.3012953761616438
0.03 0.4746953451131265
0.1 0.6348240644894162
0.25 0.6784224446048986
0.3 0.6827629205595998
0.35 0.6855501867465028
0.37 0.6879845625829613
0.4 0.6880259983225367
0.43 0.683851102696139
0.5 0.6797056139144931
0.6 0.6688018820701156
0.7 0.6511178167749042
1 0.06231884057971015


In [94]:
model.save_pretrained('models/roberta_clf_ft_plus_pseudolabels')

In [95]:
pred = trainer.predict(final_test_dataset)

In [96]:
threshold = 0.6
preds = []
for text, pr in zip(final_test.text, pred.predictions):
    proba = np.exp(pr[pr[:, 0]!=-100])
    proba /= proba.sum(axis=1, keepdims=True)
    labels = (proba[:, 1] >= threshold).astype(int).tolist()
    preds.append(labels2spans(text, labels, tokenizer))
print(len(preds))

2000


In [97]:
with open("results/roberta_ft_pseudol_th06.txt", "w") as out:
    for i in range(len(preds)):
        out.write(f"{str(i)}\t{str(preds[i])}\n")

In [109]:
row = final_test.sample(1).iloc[0]
display_spans(preds[row.name], row.text)

# Train to fine-tune after training a classic classifier

In [34]:
clf_model = RobertaForSequenceClassification.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [45]:
for param in clf_model.roberta.parameters():
    param.requires_grad = False

In [47]:
training_args = TrAr(
    output_dir='./models/roberta_clf_2',   # output directory
    overwrite_output_dir=True,
    num_train_epochs=1,             # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,  # batch size for evaluation
    warmup_steps=3000,              # number of warmup steps for learning rate scheduler
    weight_decay=1e-7,              # strength of weight decay
    learning_rate=1e-3,
    logging_dir='./logs',           # directory for storing logs
    logging_steps=500,
    eval_steps=500,
    evaluation_strategy='steps',
)

In [48]:
trainer = Trainer(
    model=clf_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                      # training arguments, defined above
    train_dataset=clf_train_dataset,         # training dataset
    eval_dataset=clf_test_small_dataset,           # evaluation dataset
    tokenizer=tokenizer,
)

In [49]:
trainer.train()

Step,Training Loss,Validation Loss,Runtime,Samples Per Second
500,0.550800,0.429195,16.024700,187.211000
1000,0.515600,0.431466,16.049300,186.924000
1500,0.516900,0.429980,16.057200,186.832000
2000,0.537600,0.438840,16.056800,186.837000
2500,0.518100,0.447410,16.050800,186.906000
3000,0.528600,0.428593,16.056100,186.844000
3500,0.540700,0.445084,16.056400,186.841000
4000,0.512600,0.479808,16.064800,186.744000
4500,0.520200,0.467433,16.056800,186.837000
5000,0.528900,0.421904,16.055200,186.855000


TrainOutput(global_step=20193, training_loss=0.5100492591614214, metrics={'train_runtime': 2255.1964, 'train_samples_per_second': 8.954, 'total_flos': 52054189856424960, 'epoch': 1.0})

In [70]:
for param in clf_model.roberta.parameters():
    param.requires_grad = True

In [75]:
training_args = TrAr(
    output_dir='./models/roberta_clf_2',   # output directory
    overwrite_output_dir=True,
    num_train_epochs=1,             # total # of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=64,  # batch size for evaluation
    warmup_steps=5000,              # number of warmup steps for learning rate scheduler
    weight_decay=1e-8,              # strength of weight decay
    learning_rate=1e-6,
    logging_dir='./logs',           # directory for storing logs
    logging_steps=500,
    eval_steps=500,
    evaluation_strategy='steps',
)

In [76]:
trainer = Trainer(
    model=clf_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                      # training arguments, defined above
    train_dataset=clf_train_dataset,         # training dataset
    eval_dataset=clf_test_small_dataset,           # evaluation dataset
    tokenizer=tokenizer,
)

In [77]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [78]:
trainer.train()

Step,Training Loss,Validation Loss,Runtime,Samples Per Second
500,0.275800,0.283414,16.231500,184.826000
1000,0.277800,0.286870,16.110000,186.220000
1500,0.247400,0.300099,16.116500,186.145000
2000,0.249300,0.315221,16.118100,186.126000
2500,0.238100,0.341812,16.116100,186.149000
3000,0.201500,0.361610,16.117700,186.131000
3500,0.211500,0.372231,16.119700,186.108000
4000,0.190900,0.380558,16.115500,186.156000
4500,0.200000,0.396305,16.115900,186.151000
5000,0.196700,0.411312,16.110900,186.210000


RuntimeError: write(): fd 126 failed with No space left on device

In [80]:
clf_model.save_pretrained('models/roberta_clf_2')

### fine tune

In [81]:
clf_model = None

In [82]:
model = AutoModelForTokenClassification.from_pretrained('models/roberta_clf_2')

Some weights of the model checkpoint at models/roberta_clf_2 were not used when initializing RobertaForTokenClassification: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at models/roberta_clf_2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

In [83]:
for param in model.roberta.parameters():
    param.requires_grad = False

In [88]:
training_args = TrAr(
    output_dir='./models/roberta_ft_v2',   # output directory
    overwrite_output_dir=True,
    num_train_epochs=10,            # total # of training epochs
    per_device_train_batch_size=64,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=3000,               # number of warmup steps for learning rate scheduler
    weight_decay=1e-8,              # strength of weight decay
    learning_rate=1e-3,
    logging_dir='./logs',           # directory for storing logs
    logging_steps=500,
    eval_steps=500,
    evaluation_strategy='steps',
    save_total_limit=3,
)

In [89]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset,           # evaluation dataset
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [90]:
trainer.train()

Step,Training Loss,Validation Loss,Runtime,Samples Per Second
500,0.218200,0.181255,2.022100,341.236000
1000,0.218100,0.183458,2.053600,335.999000


TrainOutput(global_step=1250, training_loss=0.21767515563964843, metrics={'train_runtime': 376.8704, 'train_samples_per_second': 3.317, 'total_flos': 13112992679461536, 'epoch': 10.0})

In [91]:
for param in model.roberta.parameters():
    param.requires_grad = True

In [92]:
training_args = TrAr(
    output_dir='./models/roberta_ft_v2',   # output directory
    overwrite_output_dir=True,
    num_train_epochs=10,            # total # of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=3000,               # number of warmup steps for learning rate scheduler
    weight_decay=1e-8,              # strength of weight decay
    learning_rate=1e-5,
    logging_dir='./logs',           # directory for storing logs
    logging_steps=500,
    eval_steps=500,
    evaluation_strategy='steps',
    save_total_limit=3,
)

In [93]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset,           # evaluation dataset
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [94]:
trainer.train()

Step,Training Loss,Validation Loss,Runtime,Samples Per Second
500,0.210100,0.182794,2.021700,341.304000
1000,0.231000,0.175530,2.016800,342.121000
1500,0.220400,0.175073,2.015900,342.272000
2000,0.214800,0.175506,2.011800,342.979000
2500,0.212400,0.179223,2.013500,342.688000
3000,0.214300,0.186044,2.017800,341.949000
3500,0.206600,0.170099,2.017900,341.937000
4000,0.198100,0.186814,2.020800,341.442000
4500,0.179500,0.180094,2.020600,341.478000
5000,0.179900,0.204089,2.018300,341.879000


KeyboardInterrupt: 

In [95]:
pred = trainer.predict(eval_dataset)
for threshold in [0, 0.01, 0.03, 0.1, 0.25, 0.3, 0.35, 0.37, 0.4, 0.43, 0.5, 0.6, 0.7, 1]:
    preds = []
    for text, pr in zip(trial.text, pred.predictions):
        proba = np.exp(pr[pr[:, 0]!=-100])
        proba /= proba.sum(axis=1, keepdims=True)
        labels = (proba[:, 1] >= threshold).astype(int).tolist()
        preds.append(labels2spans(text, labels, tokenizer))
    print(threshold, np.mean([f1(p, y) for p, y in zip(preds, trial.spans)]))

0 0.21345926240649193
0.01 0.4157790976472464
0.03 0.5294728448610527
0.1 0.6148151396215593
0.25 0.6550136242080651
0.3 0.6646029995260339
0.35 0.6715131454982759
0.37 0.6733516869972194
0.4 0.6744209810749072
0.43 0.6741804277228053
0.5 0.673925228987182
0.6 0.6717334585591548
0.7 0.6682677734353392
1 0.06231884057971015


Вывод: лучше вроде бы не стало, хоть и сильно хуже - тоже. 

In [96]:
model.save_pretrained('models/roberta_ft_v2')